#P3
https://github.com/rizafahmi/pytorch-demo-basic/blob/master/11train.py

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.datasets import load_iris

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode the target variable for multi-class classification
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = torch.from_numpy(scaler.fit_transform(X_train)).float()
X_test = torch.from_numpy(scaler.transform(X_test)).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

# Define the neural network architecture
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 3)  # Multi-class classification with 3 classes
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.softmax(self.layer3(x))
        return x

model = IrisModel()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model with Gradient Descent
print("Training with Gradient Descent...")
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, torch.max(y_train, 1)[1])
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}/{10}, Loss: {loss.item()}')

# Train the model with Stochastic Gradient Descent (SGD)
print("\nTraining with Stochastic Gradient Descent (SGD)...")
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(50):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, torch.max(y_train, 1)[1])
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}/{50}, Loss: {loss.item()}')


#P4
https://github.com/PacktPublishing/Deep-learning-with-PyTorch-video/blob/master/3.cnn.for.digits.classification.ipynb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 3 * 3, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
for epoch in range(5):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f'Test accuracy: {test_accuracy}')

#P5
https://www.geeksforgeeks.org/time-series-forecasting-using-pytorch/

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler  # Add this line
import matplotlib.pyplot as plt

# Rest of your code...


# Function to fetch historical stock data
def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

# Function to preprocess stock data
def preprocess_data(stock_data):
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Extracting the closing prices
    closing_prices = stock_data['Close'].values.reshape(-1, 1)

    # Scaling the closing prices
    scaled_prices = scaler.fit_transform(closing_prices)

    return scaled_prices, scaler

# Function to create sequences for training the GRU model
def create_sequences(data, sequence_length):
    sequences, labels = [], []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        label = data[i+sequence_length:i+sequence_length+1]
        sequences.append(seq)
        labels.append(label)
    return torch.Tensor(sequences), torch.Tensor(labels)

# GRU model definition
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, h_n = self.gru(x)
        out = self.fc(h_n[-1])
        return out

# Function to plot actual vs predicted prices
def plot_predictions(actual, predicted, ticker):
    plt.figure(figsize=(10, 6))
    plt.plot(actual, label='Actual Prices', color='blue')
    plt.plot(predicted, label='Predicted Prices', color='red')
    plt.title(f'{ticker} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()

# Main function
def main():
    ticker = 'AAPL'
    start_date = '2022-01-01'
    end_date = '2023-01-01'
    sequence_length = 10  # Number of previous days to consider for prediction

    # Fetch historical stock data
    stock_data = fetch_stock_data(ticker, start_date, end_date)

    # Preprocess the data
    scaled_prices, scaler = preprocess_data(stock_data)

    # Create sequences for training the model
    X, y = create_sequences(scaled_prices, sequence_length)

    # Split the data into training and testing sets
    split_ratio = 0.8
    split_index = int(len(X) * split_ratio)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Define PyTorch DataLoader for training and testing sets
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # Model parameters
    input_size = 1
    hidden_size = 50
    output_size = 1

    # Build and train the GRU model
    model = GRUModel(input_size, hidden_size, output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 50
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Predict on the test set
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predictions.append(outputs.numpy())

    predicted_prices = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    actual_prices = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))

    # Plot the results
    plot_predictions(actual_prices, predicted_prices, ticker)

if __name__ == "__main__":
    main()


#P 1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.sigmoid(x)
        x = self.layer2(x)
        return x

# Set random seed for reproducibility
torch.manual_seed(42)

# Instantiate the model
input_size = X_train.shape[1]
hidden_size = 5  # You can adjust this based on your problem
output_size = 3  # Number of classes in the Iris dataset

model = SimpleNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training the model
num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    y_pred = model(X_test)
    _, predicted = torch.max(y_pred, 1)

accuracy = accuracy_score(y_test, predicted.numpy())
print(f'Accuracy on the test set: {accuracy * 100:.2f}%')

Epoch [100/1000], Loss: 0.3645
Epoch [200/1000], Loss: 0.1520
Epoch [300/1000], Loss: 0.0903
Epoch [400/1000], Loss: 0.0704
Epoch [500/1000], Loss: 0.0614
Epoch [600/1000], Loss: 0.0565
Epoch [700/1000], Loss: 0.0535
Epoch [800/1000], Loss: 0.0516
Epoch [900/1000], Loss: 0.0504
Epoch [1000/1000], Loss: 0.0495
Accuracy on the test set: 100.00%


#P 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y)  # For multi-class classification

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define the neural network model
class DeepNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(DeepNeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.activation1 = nn.Tanh()  # Tanh activation function for the first hidden layer
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.activation2 = nn.ReLU()  # ReLU activation function for the second hidden layer
        self.output_layer = nn.Linear(hidden_size2, output_size)
        self.output_activation = nn.Softmax(dim=1)  # Softmax activation for multi-class classification

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation1(x)
        x = self.layer2(x)
        x = self.activation2(x)
        x = self.output_layer(x)
        x = self.output_activation(x)
        return x

# Instantiate the model
input_size = X.shape[1]
hidden_size1 = 64
hidden_size2 = 32
output_size = len(iris.target_names)  # Number of classes in the Iris dataset

model = DeepNeuralNetwork(input_size, hidden_size1, hidden_size2, output_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss is suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
NUM_EPOCHS = 50

for epoch in range(NUM_EPOCHS):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2%}')

Epoch [1/50], Loss: 1.0674, Accuracy: 70.00%
Epoch [2/50], Loss: 1.0381, Accuracy: 73.33%
Epoch [3/50], Loss: 0.9907, Accuracy: 76.67%
Epoch [4/50], Loss: 0.9631, Accuracy: 76.67%
Epoch [5/50], Loss: 0.8943, Accuracy: 76.67%
Epoch [6/50], Loss: 0.8945, Accuracy: 76.67%
Epoch [7/50], Loss: 0.8068, Accuracy: 80.00%
Epoch [8/50], Loss: 0.8746, Accuracy: 80.00%
Epoch [9/50], Loss: 0.8454, Accuracy: 76.67%
Epoch [10/50], Loss: 0.7906, Accuracy: 76.67%
Epoch [11/50], Loss: 0.7609, Accuracy: 80.00%
Epoch [12/50], Loss: 0.7934, Accuracy: 80.00%
Epoch [13/50], Loss: 0.8349, Accuracy: 80.00%
Epoch [14/50], Loss: 0.7516, Accuracy: 80.00%
Epoch [15/50], Loss: 0.7523, Accuracy: 83.33%
Epoch [16/50], Loss: 0.7861, Accuracy: 83.33%
Epoch [17/50], Loss: 0.7401, Accuracy: 86.67%
Epoch [18/50], Loss: 0.7926, Accuracy: 86.67%
Epoch [19/50], Loss: 0.7431, Accuracy: 86.67%
Epoch [20/50], Loss: 0.7925, Accuracy: 93.33%
Epoch [21/50], Loss: 0.6941, Accuracy: 93.33%
Epoch [22/50], Loss: 0.7281, Accuracy: 93.3

#P 7


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

# Set random seed for reproducibility
torch.manual_seed(42)

# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 20

# Define the denoising autoencoder model
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        decoded = decoded.view(decoded.size(0), 1, 28, 28)  # Reshape to image size
        return encoded, decoded

# Load MNIST dataset using torchvision
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x + 0.3 * torch.randn_like(x))  # Add random noise
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = DenoisingAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        _, reconstructed = model(data)
        loss = criterion(reconstructed, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Print progress
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader.dataset):.4f}')

# Save the trained model
if not os.path.exists('models'):
    os.makedirs('models')

torch.save(model.state_dict(), 'models/denoising_autoencoder.pth')

# Test the model by reconstructing some images
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=5, shuffle=True)

model.eval()
with torch.no_grad():
    for data, _ in test_loader:
        _, reconstructed = model(data)
        original_images = data.view(-1, 1, 28, 28)
        reconstructed_images = reconstructed.view(-1, 1, 28, 28)
        comparison = torch.cat([original_images, reconstructed_images])
        save_image(comparison, 'reconstructed_images.png', nrow=5)
        break  # Only visualize the first batch

print('Reconstructed images saved as reconstructed_images.png')


100%|██████████| 9912422/9912422 [00:00<00:00, 90928142.83it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 90399771.51it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 153325309.20it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 21672956.51it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch [1/20], Loss: 0.0012
Epoch [2/20], Loss: 0.0010
Epoch [3/20], Loss: 0.0009
Epoch [4/20], Loss: 0.0009
Epoch [5/20], Loss: 0.0009
Epoch [6/20], Loss: 0.0009
Epoch [7/20], Loss: 0.0008
Epoch [8/20], Loss: 0.0008
Epoch [9/20], Loss: 0.0008
Epoch [10/20], Loss: 0.0008
Epoch [11/20], Loss: 0.0008
Epoch [12/20], Loss: 0.0008
Epoch [13/20], Loss: 0.0008
Epoch [14/20], Loss: 0.0008
Epoch [15/20], Loss: 0.0008
Epoch [16/20], Loss: 0.0008
Epoch [17/20], Loss: 0.0008
Epoch [18/20], Loss: 0.0008
Epoch [19/20], Loss: 0.0008
Epoch [20/20], Loss: 0.0008
Reconstructed images saved as reconstructed_images.png


In [ ]:
!pip install torch --upgrade
!pip install torchvision --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.9 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu121
    Uninstalling torchvision-0.16.0+cu121:
      Successfully uninstalled torchvision-0.16.0+cu121


#P8

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from keras.datasets import imdb
from keras.preprocessing import sequence
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)

# Load IMDB dataset from Keras
max_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)

# Pad sequences to a fixed length
max_len = 200
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

# Convert data to PyTorch tensors
x_train_tensor = torch.from_numpy(x_train).long()
y_train_tensor = torch.from_numpy(y_train).float()
x_test_tensor = torch.from_numpy(x_test).long()
y_test_tensor = torch.from_numpy(y_test).float()

# Create DataLoader
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define LSTM Model
class SentimentAnalysisLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, output_size):
        super(SentimentAnalysisLSTM, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1, :, :])
        out = self.sigmoid(out)
        return out

# Instantiate the model
input_size = max_words
embedding_size = 128
hidden_size = 100
num_layers = 1
output_size = 1

model = SentimentAnalysisLSTM(input_size, embedding_size, hidden_size, num_layers, output_size)

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 5
for epoch in range(num_epochs):
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x)
        predicted = (outputs.squeeze() > 0.5).float()
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')



# Load IMDB dataset from Keras
max_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)

# Get word index
word_index = imdb.get_word_index()
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2

# Reverse the word index
reverse_word_index = {v: k for k, v in word_index.items()}

# Sample input sentence
sample_sentence = "The movie was great, and I really enjoyed it!"

# Tokenize and pad the input sentence
sample_sequence = [word_index[word] if word in word_index and word_index[word] < max_words else 0 for word in sample_sentence.split()]
sample_sequence = sequence.pad_sequences([sample_sequence], maxlen=max_len)

# Convert the sequence to a PyTorch tensor
sample_tensor = torch.from_numpy(np.array(sample_sequence)).long()

# Make a prediction using the trained model
model.eval()
with torch.no_grad():
    prediction = model(sample_tensor)
    sentiment = "Positive" if prediction.item() > 0.5 else "Negative"

print(f"Input Sentence: {sample_sentence}")
print(f"Predicted Sentiment: {sentiment} (Probability: {prediction.item():.4f})")


17464789/17464789 [==============================] - 2s 0us/step
Epoch [1/5], Loss: 0.5635
Epoch [2/5], Loss: 0.3502
Epoch [3/5], Loss: 0.3741
Epoch [4/5], Loss: 0.2994
Epoch [5/5], Loss: 0.2035
Test Accuracy: 0.8521


#P6

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from keras.datasets import imdb
from keras.preprocessing import sequence

# Set random seed for reproducibility
torch.manual_seed(42)

# Hyperparameters
embedding_size = 128
hidden_size = 256
num_layers = 2
num_epochs = 5
learning_rate = 0.001
batch_size = 64

# Load IMDb dataset from Keras
vocab_size = 20000  # Consider top 20,000 words in the dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to make them of equal length
X_train = sequence.pad_sequences(X_train, maxlen=200)
X_test = sequence.pad_sequences(X_test, maxlen=200)

# Convert to PyTorch tensors
X_train = torch.from_numpy(X_train).long()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).long()
y_test = torch.from_numpy(y_test).float()

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the output of the last time step
        return out

# Instantiate the model
model = LSTMModel(vocab_size, embedding_size, hidden_size, num_layers)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        inputs = Variable(X_train[i:i+batch_size])
        labels = Variable(y_train[i:i+batch_size])

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(X_train), loss.item()))

# Evaluate the model on test data
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(0, len(X_test), batch_size):
        inputs = Variable(X_test[i:i+batch_size])
        labels = Variable(y_test[i:i+batch_size])

        outputs = model(inputs)
        predicted = (torch.sigmoid(outputs) > 0.5).float()

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test data: {} %'.format(100 * correct / total))

# Demonstrate text generation (generate text based on a seed input)
seed_text = "This movie was"
seed_sequence = [word_index[word] if word in word_index else 0 for word in seed_text.split()]
seed_sequence = torch.tensor(seed_sequence).view(1, -1)

# Generate text using the trained model
generated_text = seed_text
model.eval()
with torch.no_grad():
    for _ in range(50):  # Generate 50 words
        outputs = model(seed_sequence)
        predicted_word_index = torch.argmax(outputs).item()
        generated_text += " " + index_word[predicted_word_index]
        seed_sequence = torch.cat((seed_sequence[:, 1:], torch.tensor([[predicted_word_index]])), dim=1)

print("Generated Text: ", generated_text)


#P7 Alternative

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import numpy as np
import matplotlib.pyplot as plt

# Define the denoising autoencoder model in PyTorch
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Load MNIST dataset and add noise
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

def add_noise(img, noise_factor=0.5):
    noise = torch.randn_like(img) * noise_factor
    noisy_img = img + noise
    return torch.clamp(noisy_img, 0., 1.)

# Initialize the denoising autoencoder model
model = DenoisingAutoencoder()

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the denoising autoencoder
num_epochs = 10
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for epoch in range(num_epochs):
    model.train()
    for data in train_loader:
        img, _ = data
        noisy_img = add_noise(img)

        optimizer.zero_grad()
        outputs = model(noisy_img)
        loss = criterion(outputs, img)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Denoise some test images
model.eval()
denoised_images = []
with torch.no_grad():
    for data in test_loader:
        img, _ = data
        noisy_img = add_noise(img)
        outputs = model(noisy_img)
        denoised_images.append(outputs)

# Display some examples
n = 10  # Number of images to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original images
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(noisy_img[i].squeeze().numpy(), cmap='gray')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display denoised images
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(denoised_images[0][i].squeeze().numpy(), cmap='gray')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

#P2 Alternative

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate some random data for demonstration purposes
torch.manual_seed(42)
X = torch.rand((1000, 10))
y = torch.randint(2, (1000, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y.numpy(), test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = torch.from_numpy(scaler.fit_transform(X_train)).float()
X_test = torch.from_numpy(scaler.transform(X_test)).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

# Define the neural network architecture
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 1)
        self.layer2 = nn.Linear(1, 64)
        self.layer3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, 1)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.tanh(x)
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.sigmoid(self.layer4(x))
        return x

model = CustomModel()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Train the model
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    test_accuracy = ((test_outputs > 0.5) == y_test.byte()).float().mean()

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {test_accuracy.item()}')


#P1 Alternative

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate some random data for demonstration purposes
torch.manual_seed(42)
X = torch.rand((1000, 10))
y = torch.randint(2, (1000, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y.numpy(), test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = torch.from_numpy(scaler.fit_transform(X_train)).float()
X_test = torch.from_numpy(scaler.transform(X_test)).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x

model = SimpleNN()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Train the model
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    test_accuracy = ((test_outputs > 0.5) == y_test.byte()).float().mean()

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {test_accuracy.item()}')
